In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/Evolution_of_AI.pdf")
pages = loader.load_and_split()
print(len(pages))
pages[:5]

15


[Document(metadata={'source': '../data/Evolution_of_AI.pdf', 'page': 0}, page_content='Jackson, Emerson Abraham\nWorking Paper\nThe Evolution of Artificial Intelligence: A Theoretical\nReview of its Impact on Teaching and Learning in the\nDigital Age\nSuggested Citation: Jackson, Emerson Abraham (2024) : The Evolution of Artificial Intelligence:\nA Theoretical Review of its Impact on Teaching and Learning in the Digital Age, ZBW – Leibniz\nInformation Centre for Economics, Kiel, Hamburg\nThis Version is available at:\nhttps://hdl.handle.net/10419/280893\nStandard-Nutzungsbedingungen:\nDie Dokumente auf EconStor dürfen zu eigenen wissenschaftlichen\nZwecken und zum Privatgebrauch gespeichert und kopiert werden.\nSie dürfen die Dokumente nicht für öffentliche oder kommerzielle\nZwecke vervielfältigen, öffentlich ausstellen, öffentlich zugänglich\nmachen, vertreiben oder anderweitig nutzen.\nSofern die Verfasser die Dokumente unter Open-Content-Lizenzen\n(insbesondere CC-Lizenzen) zur Ver

In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [6]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key = os.getenv('GEMINI_API'))

# This function takes a a sentence as an arugument and return it's embeddings
def get_embeddings(text):
    # Define the embedding model
    model = 'models/embedding-001'
    # Get the embeddings
    embedding = genai.embed_content(model=model, content=text, task_type="retrieval_document")
    return embedding['embedding']

# Get the page_content from the documents and create a new list
content_list = [doc.page_content for doc in pages]
content_list

# Send one page_content at a time
embeddings = [get_embeddings(content) for content in content_list]
embeddings

[[0.038449973,
  -0.060039613,
  -0.030532485,
  -0.0148638245,
  0.035070885,
  0.007111906,
  0.012441421,
  0.007835618,
  -0.03385785,
  0.0763024,
  0.018730635,
  0.007977896,
  0.021639854,
  0.012707907,
  0.024721295,
  -0.05084241,
  0.021531045,
  0.056625947,
  -0.029838962,
  -0.012806325,
  0.027264416,
  0.019229053,
  0.057374906,
  -0.006776519,
  -0.019083118,
  -0.00554767,
  -0.0096666245,
  -0.04887937,
  -0.016818637,
  -0.01066661,
  -0.053046595,
  0.0010696489,
  -0.039318804,
  0.027298903,
  -0.001201089,
  -0.03320162,
  0.0009947995,
  0.049470548,
  0.013570248,
  0.02148047,
  -0.029077966,
  -0.054651737,
  -0.0324016,
  -0.009315923,
  -0.005146895,
  -0.052793786,
  0.04020916,
  0.08763287,
  0.043757945,
  -0.058093328,
  0.030958205,
  0.046031676,
  0.049117677,
  -0.036982417,
  0.057161704,
  -0.044648483,
  0.06853029,
  0.031677797,
  -0.011763538,
  -0.01996933,
  -0.028681785,
  0.0025028773,
  -0.046927247,
  0.029844606,
  0.0010839205,
  -

In [22]:
type(embeddings)

list

In [7]:
import chromadb

# Initialize ChromaDB client
client = chromadb.Client()
collection = client.create_collection('RAG_Pipeline')

In [19]:
# Set up a retriever with top_k=5
results = collection.query(query_embeddings = embeddings, n_results = 5)

In [24]:
results

{'ids': [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []],
 'embeddings': None,
 'documents': [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []],
 'uris': None,
 'data': None,
 'metadatas': [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []],
 'distances': [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [21]:
context = "\n".join([doc['text'] for doc in results])
context

TypeError: string indices must be integers, not 'str'

In [16]:
api_key = os.getenv('GEMINI_API')

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key = api_key)

In [18]:
from langchain.prompts import PromptTemplate


from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Create the retrieval chain
template = """
You are a helpful AI assistant.
Answer based on the context provided. 
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

# Invoke the retrieval chain
response = retrieval_chain.invoke({"input": "How do I apply for personal leave?"})

# Print the answer to the question
print(response["answer"])


AttributeError: 'dict' object has no attribute 'with_config'